In [7]:
%matplotlib inline 
from matplotlib import pyplot as plt

import json
import time
import random
import logging
import datetime
import threading
from paho import mqtt
from time import sleep
import multiprocessing
from threading import Thread
from time import perf_counter
import paho.mqtt.client as paho
from IPython.display import clear_output
from scipy.spatial import distance as dist
from imutils.video import FileVideoStream
from imutils.video import VideoStream
from pynq.lib.video import *
from imutils import face_utils
import multiprocessing
import numpy as np  
import argparse
import platform
import imutils
import dlib
import cv2
import os

# 1. Creacion Ficheros de Log

In [8]:
# Definición del logger root
# -----------------------------------------------------------------------------
logging.basicConfig(
    format = '%(asctime)-5s %(name)-15s %(levelname)-8s %(message)s',
    level  = logging.INFO,
    filemode = "a"
    )
# Nuevos handlers
# -----------------------------------------------------------------------------
# Si el root logger ya tiene handlers, se eliminan antes de añadir los nuevos.
# Esto es importante para que los logs no empiezen a duplicarse.
if logging.getLogger('').hasHandlers():
    logging.getLogger('').handlers.clear()
    
# Se añaden tres nuevos handlers al root logger, uno para los niveles de debug o
# superiores, otro recogida de datos y otro para que se muestre por pantalla los niveles de info o
# superiores.
#---------------Recogida de datos de eventos---------------------

log_data= f"logs_data_pynq_{datetime.datetime.now().strftime('%Y%m%d.log')}"
file_data_handler = logging.FileHandler(log_data)
 
file_data_handler.setLevel(logging.CRITICAL)
file_data_format = logging.Formatter('%(asctime)-5s %(name)-15s %(levelname)-8s %(message)s')
file_data_handler.setFormatter(file_data_format)
logging.getLogger('').addHandler(file_data_handler)

#-------------------Trazas de Debug del prototipo -----------------

log_debg = f"logs_debug_{datetime.datetime.now().strftime('%Y%m%d_test.log')}"
file_debug_handler = logging.FileHandler(log_debg)
#file_debug_handler .TimedRotatingFileHandler(filename = log_debg,  when='s', interval=1, backupCount=5)
 
file_debug_handler.setLevel(logging.DEBUG)
file_debug_format = logging.Formatter('%(asctime)-5s %(name)-15s %(levelname)-8s %(message)s')
file_debug_handler.setFormatter(file_debug_format)
logging.getLogger('').addHandler(file_debug_handler)

#----------------Trazas de infor para desarrollo---------------------

consola_handler = logging.StreamHandler()
consola_handler.setLevel(logging.INFO)
consola_handler_format = logging.Formatter('%(asctime)-5s %(name)-15s %(levelname)-8s %(message)s')
consola_handler.setFormatter(consola_handler_format)
logging.getLogger('').addHandler(consola_handler) 

# 2. Comunicación MQTT

## 2.1 constantes

In [9]:
#--------------------------------------------------------------
# Definición Constantes y Mensajes Intercambio HiveMQtt e UART/Bluetooth
#----------------------------------------------------------------
HOST_HIVEMQ   = "dbd25372b1b647e895c3f4adb0260f4e.s2.eu.hivemq.cloud" 
PORT_HIVEMQ   = 8883
clean_session = True

client_id     = "Client_00"
server_id     = "Client_01"
client_cam_id = "Client_03"

user_name     = "adasjg01"
password      = "adas2022_01"

user_name2     = "adasjg01"
password2      = "adasjg02_iot02"

user_name_cam     = "adasjg_cam"
password_cam      = "adas2022_cam"

topic_sub_pynq       = "ADASJG01/Pynq/#" # De esta forma leo los mensajes de intercambio y los eento de la cam
topic_pub_pynq       = "ADASJG01/Server/"
topic_pub_client_cam = "ADASJG01/Server/cam/"

## 2.2 Publicador eventos de somnolencia

In [10]:
# MQTT: client_cam
client_cam = None
send_cam_scr = threading.Semaphore()  # Para publicar mesajes mediante diferentes hilos en exclusión y con sección critica (la publicación)

#--------------------------------------------------------
# Publicador eventos CAM
#------------------------------------------------------------
# Desarrollo multithreading, aparecen secciones criticas en los 
# adaptadores, publicadores y zonas compartidas. Mejora la respuesta 
# con el proceso y análisis de vídeo. Permite tener hilos dedicados 
# a cada tipo de comunicación (mqtt,uart, etc...)
def on_connect_client_cam (client, userdata, flags, rc, properties=None):
    if rc == 0:
        #logging.debug ("CLient_Cam::>>> OK connect")
        client.connected_flag = True
        return
    logging.debug ("CLient_Cam::>>> Failed to connect")

    # with this callback you can see if your publish was successful
def on_publish_client_cam (client, userdata, mid, properties=None):
    print("CLient_Cam ::>>> mid: " + str(mid))

def connect_client_cam ():

    """
    Conecta topic 'ADASJG01/Pynq/cam'
    aunque con implementar una función de conexión sobraría (pasando como parametro el client_id)
    vamos a implementar dos funciones por separado para identificar claramente los dos clientes 
    que se tienen sobre el mismo broker mqtt
    """
    logging.info("connect_client_cam")
    # using MQTT version 5 here, for 3.1.1: MQTTv311, 3.1: MQTTv31
    # userdata is user defined data of any type, updated by user_data_set()
    # client_id is the given name of the client
    client = paho.Client (client_id=client_cam_id, userdata=None, protocol=paho.MQTTv5)
    client.on_connect = on_connect_client_cam
    # enable TLS for secure connection
    client.tls_set (tls_version=mqtt.client.ssl.PROTOCOL_TLS)
    # set username and password
    client.username_pw_set (user_name_cam, password_cam)
    # connect to HiveMQ Cloud on port 8883 (default for MQTT)
    client.connect (HOST_HIVEMQ, PORT_HIVEMQ, keepalive = 60)
    client.connected_flag = False
    
    while not client.connected_flag: #wait in loop
        client.loop()#client_cam.loop_start()
        sleep (1)
        logging.info ("NO CONECTA")
    logging.info ("OK CONECT")
    
    return client
def disconnect_client_cam (client):
    logging.info("disconnect_client_cam")
    send_cam_scr.acquire()
    if 'client_cam' in globals():
        client.connected_flag = False
        client.disconnect  ()
        client.unsubscribe(topic_pub_client_cam)
        client.loop_stop ()
    send_cam_scr.release()
#------------
# send_cloud_cam_mseg (topic,msg_out): Envia los eventos de parpadeo/bostezo/Somnolencia a la pynq
# Realement eenvia las accciones que debe realiar ante dichas situaciones.
# Al igual que con la función de conexión. Vamos a crear dos funciones de publicación por cliente (hilo)
# realmente con una sóla implementación sobraría (pasas como parametro la instancia de tu cliente)
# ----------------------------------------------------------------------------------------------------------------
# Nota: Aunque en la implementación sele pasa como aributo 'client', Debemos tomar esta función y 
# por tanto el objeto 'client_cam' como sección crítica. Ya que se lanan varios hilos y'client_cam' es global
# a la hora de compartir dicho objeto, puedo producirse un error de carrera si varios hilos modifican y usan a la vez
# client_cam para publicar. Ya se ha testeado que se roduce un error si no se sincroniza en exclusión mutua dicha función
# por ello, añadimos el objeto 'client_cam' dentro de la función y así reducimos la sección crñitica en exclusión mútua.
#---------------
def send_cloud_cam_mseg (client,msg_out):
    #Sección crítica, par aevitar problemas de multithreding a la hora de enviar notficaciones
    #por eventos de somnolencia
    
    logging.debug("send_cloud_cam_mseg")
    send_cam_scr.acquire()
    if 'client_cam' in globals():
        logging.debug(msg_out)
        print("send_cloud_cam_mseg")
        if msg_out is not None and len (msg_out) > 0:
            for i in range(0,len(msg_out)):
                msg = msg_out [i]
                client_cam.publish (topic_pub_client_cam, payload = msg, qos=0)
    send_cam_scr.release()    

In [11]:
client_cam = connect_client_cam ()

2023-01-05 09:02:20,297 root            INFO     connect_client_cam
2023-01-05 09:02:21,798 root            INFO     NO CONECTA
2023-01-05 09:02:21,804 root            INFO     OK CONECT


## 2.3 subscritor de la APP para la PYNQ. Publicadr Eventos de distracción

In [12]:
client_hivemqtt = None
send_src     = threading.Semaphore()  # Para compartir publicador con los hilos que publican cada evento
#####################
# Cliente recive eventos placa Pynq y envia a servidor cada evento detectado por lso sensores
# Recive al acción a realizar del servidor y efectua la acción con los actuadores necesarios a dicha acción
#####################

# setting callbacks for different events to see if it works, print the message etc.
def on_connect_pynq (client, userdata, flags, rc, properties=None):
    if rc == 0:
        #logging.info (" Client ::> Ok Connect")
        client.connected_flag = True
        return
       
    logging.info (" Client ::> Failed Connect")
    
# with this callback you can see if your publish was successful
#def on_publish_pynq (client, userdata, mid, properties=None):
   # print(" Cliente PYNQ:::>>> mid: " + str(mid))

# print which topic was subscribed to
def on_subscribe_pynq  (client, userdata, mid, granted_qos, properties=None):
    print(" Cliente PYNQ:::>>> Subscribed: " + str(mid) + " " + str(granted_qos))
 
# print message, useful cfor checking if it was successful
def on_message_pynq (client, userdata, msg):
    
    print(f' Cliente PYNQ:::>>> Thread {threading.get_ident()} - Processing on_message')
     
  
    str_msg = str(msg.payload.decode("utf-8"))
 
    if str_msg is not None:
        print("Cliente PYNQ:::>>> str_msg")
        event = recv_evet_msg_json (str_msg)
        print("event:",event)
        if event  == EVENT_LOGIN:
            logging.info ('EVENT_LOGIN')
            status    = recv_login_msg_json (str_msg)
            logging.info (status)
            if status == "OK":
                led_rgb (4,2)
            else:
                led_rgb (4,4)
        elif (get_color_led_rgb (4) ==  2 ) and event  == EVENT_LED_STICK:
            logging.info ('EVENT_LED_STICK')
            
            color = recv_params_msg_json (str_msg)
            
            print (' >>>>>>>>>>>>> color:',int(color[0]))
            led_stick (int(color[0]))
            
        elif (get_color_led_rgb (4) ==  2 ) and event  == EVENT_BUZZER:
            logging.info ('EVENT_BUZZER')
            paramms = recv_params_msg_json (str_msg)
            
            note    = paramms [0]
            beats   = [int(x) for x in paramms [1].split(sep=',')] #paramms [1]
            tempo   = int (paramms [2])
            count   = int (paramms [3])
            
            print (' >>>>>>>>>>>>> note:',note)
            print (' >>>>>>>>>>>>> beats:',beats)
            print (' >>>>>>>>>>>>> tempo:',tempo)
            print (' >>>>>>>>>>>>> count:',count)
            
            n = count
            
            for i in range(0,n):
                buzzer (note,beats,tempo)
  
        elif (get_color_led_rgb (4) ==  2 ) and event  == EVENT_SERVO:
            logging.info ('EVENT_SERVO')
            
            angle = recv_params_msg_json (str_msg)
            print (' >>>>>>>>>>>>> angle:',float(angle[0]))
            
            servo (int(angle[0]))
            
        elif (get_color_led_rgb (4) ==  2 ) and event  == EVENT_DTLIGTH:
            logging.info ('EVENT_DTLIGTH') 
            inten = recv_intensity_msg_json (str_msg)
            logging.info (f'Se ha detectado un aluz interna con una intensidad de {inten}') 
            logging.info (f'Recuerde que por llevar luces internas encendidas le pueden multar con 200 euros.') 
        elif (get_color_led_rgb (4) ==  2 ) and event  == EVENT_DTTEMP:
            logging.info ('EVENT_DTTEMP')
            temp = recv_temp_msg_json (str_msg)
            logging.info (f'Se ha detectado una temperatura de {temp}ºC')   
        else:
            logging.info('error')
    else:
        print("Cliente PYNQ:::>>> NOT str_msg")
 
def subscribing_pynq    (client,topic):
    client.on_message   = on_message_pynq
    client.on_subscribe = on_subscribe_pynq  
    client.subscribe (topic, qos=0)
    client.loop_forever()
        
def main_pynq (client,topic,msg):
    send_src.acquire()
     # a single publish, this can also be done in loops, etc.
    client.publish (topic, payload = msg, qos=0)
    send_src.release()
        
#----------------------------
def disconnect_hivemqtt(client):
    logging.info ("disconnect_hivemqtt ()")
    
    if 'client_hivemqtt' in globals() or 'client_hivemqtt' in locals():
        client.disconnect ()
        client.unsubscribe (topic_sub_pynq)
        client.connected_flag = False
        client.loop_stop ()
        

def isConnect_hivemqtt  (client):
    logging.info("isConnect_hivemqtt ()")
  
    if 'client_hivemqtt' in globals() or 'client_hivemqtt' in locals():
        return client.connected_flag 
    else:
        return False

def app_connect_hivemqtt_run ():
    logging.info ("def app_connect_hivemqtt_run ():")
    #disconnect_hivemqtt()
    
    # using MQTT version 5 here, for 3.1.1: MQTTv311, 3.1: MQTTv31
    # userdata is user defined data of any type, updated by user_data_set()
    # client_id is the given name of the client
    client = paho.Client(client_id=client_id, userdata=None, protocol=paho.MQTTv5)
    client.on_connect = on_connect_pynq 
    # enable TLS for secure connection
    client.tls_set(tls_version=mqtt.client.ssl.PROTOCOL_TLS)
    # set username and password
    client.username_pw_set(user_name, password)
    # connect to HiveMQ Cloud on port 8883 (default for MQTT)
    client.connect (HOST_HIVEMQ, PORT_HIVEMQ, keepalive = 60)
    client.connected_flag = False
    while not client.connected_flag: #wait in loop
        client.loop  ()
        sleep (1)
        
    return client

In [13]:
client_hivemqtt = app_connect_hivemqtt_run ()

2023-01-05 09:02:25,243 root            INFO     def app_connect_hivemqtt_run ():


In [14]:
#led_rgb (4,2)
#app_connect_hivemqtt_run ()
#sub_pynq = threading.Thread(target = subscribing_pynq,args=(client,topic_sub_pynq))
#sub_pynq.start ()

In [15]:
#disconnect_hivemqtt()

## 2.3 Mensajes y eventos de intercambio

In [16]:
MSG_DTC_JSON           = '{"clientid":"AGASDJG01","action":"DTC","event":"None"}'

MSG_LOGIN              = '{"clientid":"AGASDJG01","action":"DTC","event":"LOGIN","status":"OK"}'

MDG_RTC_JSON_BUZZER    =  '{"clientid":"AGASDJG01","action":"RTC", \
                             "event":"BUZZER","paramms":["a,b,c,d,e,f,g,A",\
                                                         "1, 1, 1, 1, 1, 1, 2, 1",\
                                                          "10","3"]}'
MDG_RTC_JSON_SERVO     =  '{"clientid":"AGASDJG01","action":"RTC", \
                             "event":"SERVO","paramms":["180"]}'

MDG_RTC_JSON_LED_STICK =  '{"clientid":"AGASDJG01","action":"RTC", \
                             "event":"LED_STICK","paramms":["0xFF00FF"]}' 

MSG_DTC_TEMP = '{"clientid":"AGASDJG01","action":"DTC","event":"DTTEMP","Temp":"23"}'
MSG_RTC_TEMP = '{"clientid":"AGASDJG01","action":"RTC","event":"DTTEMP","Temp":"23"}'

MSG_DTC_LIGHT_INT = '{"clientid":"AGASDJG01","action":"DTC","event":"DTLIGTH","ILight":"23.2"}'
MSG_RTC_LIGHT_INT = '{"clientid":"AGASDJG01","action":"RTC","event":"DTLIGTH","ILight":"23.2"}'

"""
'DROWSINESS'  - detección de somnolencia (cam)
'DTBLINK'     - detección parpadeo (cam)
'DTYAWN'      - detección bostezo  (cam)
'DTSMOKING'   - detección fumando  (cam)
'DTDRINK'     - detección bebiendo (cam)
'DTEATING'    - detección comiendo (cam)
'DTMOBL'      - detección movil    (cam)
'DTLIGTH'     - detección de luz interna  (sensor intensidad de luz)
'DTTEMP'      - detección temperatura     (sensor temperatura interna)
'DANGPOS'     - no se detecta a cara del conductor, bien por taparse o posición inadecuada
--------
'BUZZER'    - Creación sonido alarma
'LED_STICK' - Señalizacion mediante testigos de luz
'SERVO'     - Movimiento SERVO, simulando conducción autónoma
"""

EVENT_NFTR      = "NFTR"    # Sin implementar
EVENT_DANGPOS   = "DANGPOS"   

EVENT_LOGIN     = "LOGIN"
#-- sensor Cam
EVENT_DANGPOS    = "DANGPOS" 
EVENT_DROWSINESS = 'DROWSINESS'
EVENT_DTBLINK    = "DTBLINK"
EVENT_DTYAWN     = "DTYAWN"
EVENT_DTSMOKING  = "DTSMOKING"
EVENT_DTDRINK    = "DTDRINK"
EVENT_DTEATING   = "DTEATING"
EVENT_DTMOBIL    = 'DTMOBIL'
#-- Sensor Temp
EVENT_DTTEMP     = "DTTEMP"
#-- Sensor Intensidad de Luz
EVENT_DTLIGTH    = "DTLIGTH"

#--- Actuadores

EVENT_BUZZER    = "BUZZER"
EVENT_LED_STICK = "LED_STICK"
EVENT_SERVO     = "SERVO" 

In [17]:
import json

def msg_dangpos (clientid='Client_01'):
    msg = json.loads(MSG_DTC_JSON)
    msg ['event'] = EVENT_DANGPOS
    msg ['clientid'] = clientid
    
    return json.dumps(msg)

def msg_login (status,clientid='Client_01'):
    msg = json.loads(MSG_LOGIN)
    msg ['clientid'] = clientid
    msg ['event'] = EVENT_LOGIN
    msg ['status'] = status
 
    return json.dumps(msg)

def msg_buzzer (note,beats,tempo,count,clientid='Client_01'):
    msg = json.loads(MDG_RTC_JSON_BUZZER)
    msg ['clientid'] = clientid
    msg ['event'] = EVENT_BUZZER
    msg['paramms'][0] = note
    msg['paramms'][1] = beats
    msg['paramms'][2] = tempo
    msg['paramms'][3] = count
    
    return json.dumps(msg)

def msg_servo (angle,clientid='Client_01'):  
    msg = json.loads(MDG_RTC_JSON_SERVO)
    msg ['clientid'] = clientid
    msg ['event'] = EVENT_SERVO
    msg['paramms'][0] = angle
    
    return json.dumps(msg)

def msg_led_stick (color,clientid='Client_01'):   
    msg = json.loads(MDG_RTC_JSON_LED_STICK)
    msg ['clientid'] = clientid
    msg ['event'] = EVENT_LED_STICK
    msg['paramms'][0] = color
    
    return json.dumps(msg)

def send_msg_json (action='DTC',event=None,vparam=None,clientid='Client_01'):
    msg = None
    if vparam == None:
        msg = json.loads(MSG_DTC_JSON)
        msg ['event'] = event
    else:
        if event == EVENT_DTTEMP:
            msg = json.loads(MSG_DTC_TEMP)
            msg ['event'] = event  
            msg ['Temp']  = vparam
        elif event == EVENT_DTLIGTH:
            msg = json.loads(MSG_RTC_LIGHT_INT)
            msg ['event']  = event  
            msg ['ILight'] = vparam
        else:
            msg = json.loads(MSG_DTC_JSON)
            msg ['event'] = event
            
    if msg is not None:
        msg ['action']   = action 
        msg ['clientid'] = clientid
    json_temp = json.dumps(msg)
    
    return json_temp

def recv_evet_msg_json (msg):
    logging.debug ('recv_evet_msg_json')
    logging.debug (msg)
    str_msg = msg #str(msg.payload.decode("utf-8"))
    
    event = None
    if str_msg is not None and str_msg != '':
        msg_in  = json.loads(str_msg) #decode json data
        event = msg_in["event"]
        logging.info(event)
    else:
        logging.info ("NOT str_msg")

    return event

def recv_login_msg_json (msg):
    logging.debug('recv_login_msg_json')
    logging.debug(msg)
 
    str_msg = msg #str(msg.payload.decode("utf-8"))
    
    status = None
    if str_msg is not None:
        print("str_msg")
        msg_in  = json.loads(str_msg) #decode json data
        status = msg_in["status"]
    else:
        logging.info ("NOT str_msg")

    return status

def recv_intensity_msg_json (msg):
    logging.debug('recv_intensity_msg_json')
    logging.debug(msg)
    str_msg = msg #str(msg.payload.decode("utf-8"))
    
    paramms = None
    if str_msg is not None:
        msg_in  = json.loads(str_msg) #decode json data
        ILight = msg_in["ILight"]
        logging.debug(ILight) 
    else:
        logging.info("NOT str_msg")

    return ILight

def recv_temp_msg_json (msg):
    logging.info('recv_temp_msg_json')
    logging.debug(msg)
    str_msg = msg #str(msg.payload.decode("utf-8"))
    
    paramms = None
    if str_msg is not None:
        msg_in  = json.loads(str_msg) #decode json data
        Temp = msg_in["Temp"]
    else:
        logging.info("NOT str_msg")

    return Temp

def recv_params_msg_json (msg):
    logging.info('recv_params_msg_json')
    logging.debug(msg)
    str_msg = msg #str(msg.payload.decode("utf-8"))
    
    paramms = None
    if str_msg is not None:
        msg_in  = json.loads(str_msg) #decode json data
        paramms = msg_in["paramms"]
    else:
        logging.info ("NOT str_msg")

    return paramms 

In [18]:
print(send_msg_json (event=EVENT_DROWSINESS))
print(send_msg_json (event=EVENT_DTBLINK))
print(send_msg_json (event=EVENT_DTYAWN))
print(send_msg_json (event=EVENT_DANGPOS))
print(send_msg_json (event=EVENT_NFTR))
print(send_msg_json (event=EVENT_DTTEMP,vparam=23))
print(send_msg_json (event=EVENT_DTLIGTH,vparam=23))
print(msg_buzzer("a,b,c,d,e,f,g,A","1, 1, 1, 1, 1, 1, 2, 1","10","2"))
print(msg_led_stick (0xFF00FF))
print(msg_servo (90))
print("")

print(recv_evet_msg_json (msg_buzzer("a,b,c,d,e,f,g,A","1, 1, 1, 1, 1, 1, 2, 1","10","2")))
print(recv_params_msg_json (msg_buzzer("a,b,c,d,e,f,g,A","1, 1, 1, 1, 1, 1, 2, 1","10","3")))

print(recv_evet_msg_json (msg_servo (90)))
print(recv_params_msg_json (msg_servo (90)))
 
print(recv_evet_msg_json (msg_led_stick (0xFF00FF)))
print(recv_params_msg_json (msg_led_stick (0xFF00FF)))
 
print(recv_evet_msg_json (send_msg_json ('RTC',EVENT_DTTEMP,23 )))
print(recv_temp_msg_json (send_msg_json ('RTC',EVENT_DTTEMP,23 )))
print(recv_evet_msg_json (send_msg_json ('RTC',EVENT_DTLIGTH,23.2)))
print(recv_intensity_msg_json (send_msg_json ('RTC',EVENT_DTLIGTH,23.2)))

2023-01-05 09:02:30,452 root            INFO     BUZZER
2023-01-05 09:02:30,464 root            INFO     recv_params_msg_json
2023-01-05 09:02:30,474 root            INFO     SERVO
2023-01-05 09:02:30,480 root            INFO     recv_params_msg_json
2023-01-05 09:02:30,487 root            INFO     LED_STICK
2023-01-05 09:02:30,493 root            INFO     recv_params_msg_json
2023-01-05 09:02:30,500 root            INFO     DTTEMP
2023-01-05 09:02:30,507 root            INFO     recv_temp_msg_json
2023-01-05 09:02:30,514 root            INFO     DTLIGTH


{"clientid": "Client_01", "action": "DTC", "event": "DROWSINESS"}
{"clientid": "Client_01", "action": "DTC", "event": "DTBLINK"}
{"clientid": "Client_01", "action": "DTC", "event": "DTYAWN"}
{"clientid": "Client_01", "action": "DTC", "event": "DANGPOS"}
{"clientid": "Client_01", "action": "DTC", "event": "NFTR"}
{"clientid": "Client_01", "action": "DTC", "event": "DTTEMP", "Temp": 23}
{"clientid": "Client_01", "action": "DTC", "event": "DTLIGTH", "ILight": 23}
{"clientid": "Client_01", "action": "RTC", "event": "BUZZER", "paramms": ["a,b,c,d,e,f,g,A", "1, 1, 1, 1, 1, 1, 2, 1", "10", "2"]}
{"clientid": "Client_01", "action": "RTC", "event": "LED_STICK", "paramms": [16711935]}
{"clientid": "Client_01", "action": "RTC", "event": "SERVO", "paramms": [90]}

BUZZER
['a,b,c,d,e,f,g,A', '1, 1, 1, 1, 1, 1, 2, 1', '10', '3']
SERVO
[90]
LED_STICK
[16711935]
DTTEMP
23
DTLIGTH
23.2


# 3. Sensores /Actuadores

## 3.1 Acceso Overlay/Adapter

In [19]:
from pynq.overlays.base import BaseOverlay
 
base = BaseOverlay("base.bit")

In [20]:
from pynq_peripherals import ArduinoSEEEDGroveAdapter, PmodGroveAdapter

 

adapter  = ArduinoSEEEDGroveAdapter(base.ARDUINO, A3 = 'grove_light', 
                                    A1 = 'grove_temperature',
                                    D3 = 'grove_buzzer',
                                    D5 = 'grove_servo', 
                                    D7 = 'grove_led_stick')
 
ligth       = adapter.A3
servo       = adapter.D5
buzzer      = adapter.D3
led_stick   = adapter.D7
temp_sensor = adapter.A1

In [21]:
print('temperature: {:.2f}'.format(adapter.A1.get_temperature()))

temperature: 19.10


## 3.2 Lectura Sensores y Modificación Actuadores

In [22]:
# Secciones criticas, cuando modificacmos de forma asincrona actuadores

servo_scr     = threading.Semaphore()  #  
buzzer_src    = threading.Semaphore()  #  
led_stick_scr = threading.Semaphore()  #  
leds_rgb_scr  = threading.Semaphore()  #  

temp_scr     = threading.Semaphore()  # 
intent_scr   = threading.Semaphore()  # 
## SENSORES
def get_buttons ():
    return base.buttons.read()
def temp_sensor ():
    logging.info(' temp_sensor ( )')
    return round(adapter.A1.get_temperature(),2) #adapter.A1.get_temperature()-25

def ligth_intensity ():
    logging.info('ligth_intensity ( )')
    return adapter.A3.get_intensity()

## ACTUADORES
def get_color_led_rgb (iled):
     return base.rgbleds [iled].read ()
    
def led_rgb (iled,color):
    leds_rgb_scr.acquire()
    base.rgbleds [iled].write (color)
    leds_rgb_scr.release()
    
def led_stick (color):
    led_stick_scr.acquire()
    logging.info(' led_stick ( )')
    adapter.D7.clear()
 
    for i in range(10):
        adapter.D7.set_pixel(i, color)
        #time.sleep(1)
    adapter.D7.show()
    led_stick_scr.release()
    
def clean_led_stick ():
    led_stick_scr.acquire()
    logging.info(' clean_led_stick ( )')
    adapter.D7.clear()
    led_stick_scr.release()
        
def buzzer (note,beats,tempo):
    buzzer_src.acquire()
    logging.info(' buzzer ( )')
 

    adapter.D3.play_tone(262, 500) 
    adapter.D3.play_note('g', 500)
    from time import sleep
    notes = note#['a','b','c','d','e','f','g','A']
    beats = beats#[1, 1, 1, 1, 1, 1, 2, 1]
    tempo = tempo #20 in ms
    for [note, beat] in zip(notes, beats):
        if note==' ':
            sleep(0.001 * beat * tempo)
        else:
            adapter.D3.play_note(note,beat*tempo)
        sleep(0.001*tempo/2)
    buzzer_src.release()
    
def servo (angle):
    servo_scr.acquire()
    logging.info (' servo ()')
    print (angle)
    
    for gl in range(0, angle, 10):
        print(f"{gl} degree")
        adapter.D5.set_angular_position(gl)
        sleep(0.5)
    for gl in range(angle, 0, -10):
        print(f"{gl} degree")
        adapter.D5.set_angular_position(gl)
        sleep(0.5)
    servo_scr.release()   
    
def voice (msg):
    """
        Falta ver como añadir altavoz y sacar el mensaje
    """
    print (msg)
    
print('temperature: {:.2f}'.format(temp_sensor ()))
print('Intensidad: {:.2f}'.format(ligth_intensity ()))

2023-01-05 09:03:05,942 root            INFO      temp_sensor ( )


temperature: 19.31

2023-01-05 09:03:05,960 root            INFO     ligth_intensity ( )



Intensidad: 2.75


In [23]:
servo (10)

2023-01-05 09:03:06,005 root            INFO      servo ()


10
0 degree
10 degree


# 4. Comunicación UART/Bluetooth

## 4.1 Configuración UART

In [46]:
comando_AT_test = "AT"

comando_AT_get_name = "AT+NAME?"
## Definir comando AT+NAMEXXXXX  Ej: BT_IOT

comando_AT_get_pin = "AT+PIN?"
## Definir comando AT+PINXXXXX   Ej: 2022

comando_AT_get_bau = "AT+BAUD?"
comando_AT_SECH    = "AT+SECH?"
comando_AT_DEF     = "AT+UART_DEF?"

In [47]:
from pynq.lib import MicroblazeLibrary
UART_RXD = 0  ## Módulo BT pin 0 es RXD
UART_TXD = 1  ## Módulo BT pin 1 es TXD

lib_uart = MicroblazeLibrary(base.ARDUINO, ['uart'])
uart_scr = threading.Semaphore()  # 

In [48]:
def recv_cmd (dispositivo_uart):
    len_respuesta = 10
    respuesta = [0x00] * len_respuesta
 
    dispositivo_uart.read(respuesta,len_respuesta)  
    sleep(0.1)
    
    return respuesta

def send_cmd (dispositivo_uart, comando_at):
    comando = bytes(comando_at, 'utf-8')
    logging.info (comando_at)
    
    dispositivo_uart.write(list(comando), len(comando))
    sleep(0.3) ## Ajustable entre [0.15-0.5] ms
    
def send_synq_cmd (dispositivo_uart, comando_at):
    
    comando = bytes(comando_at, 'utf-8')
    logging.info (comando_at)
    
    dispositivo_uart.write(list(comando), len(comando))
    sleep(0.3) ## Ajustable entre [0.15-0.5] ms
    
    respuesta = recv_cmd (dispositivo_uart)
    # Mostrar respuesta en ASCII
    for i in respuesta:
        print(chr(i), end='')
        
    logging.info(respuesta)  # <list>

In [49]:
from IPython.display import clear_output
uart = lib_uart.uart_open(UART_TXD, UART_RXD)

comando_AT_test = "AT"
comando_AT_set_name = "AT+NAMEBT001_TAR"
comando_AT_set_pin = "AT+PIN2305"

send_synq_cmd(uart, comando_AT_test)

send_synq_cmd(uart, comando_AT_set_name)
send_synq_cmd(uart, comando_AT_set_pin)

send_synq_cmd(uart, comando_AT_get_bau)
send_synq_cmd(uart, comando_AT_get_pin)
send_synq_cmd(uart, comando_AT_SECH)
 
clear_output(wait=True)

2023-01-05 09:44:50,015 root            INFO     AT
2023-01-05 09:44:50,430 root            INFO     [79, 75, 0, 0, 0, 0, 0, 0, 0, 0]
2023-01-05 09:44:50,444 root            INFO     AT+NAMEBT001_TAR


OK        

2023-01-05 09:44:50,856 root            INFO     [79, 75, 43, 83, 101, 116, 58, 66, 84, 48]
2023-01-05 09:44:50,869 root            INFO     AT+PIN2305


OK+Set:BT0

2023-01-05 09:44:51,281 root            INFO     [48, 49, 95, 84, 65, 82, 79, 75, 43, 83]
2023-01-05 09:44:51,294 root            INFO     AT+BAUD?


...

et:230OK+B

2023-01-05 09:44:52,130 root            INFO     [65, 85, 68, 58, 57, 54, 79, 75, 43, 80]
2023-01-05 09:44:52,144 root            INFO     AT+SECH?


AUD:96OK+P

2023-01-05 09:44:52,555 root            INFO     [73, 78, 58, 50, 51, 48, 79, 75, 43, 83]


IN:230OK+S

## 4.2 Hilo escucha comunicación UART/Bluetooth

In [50]:
# app_connect: varaible global conexión uart/bluetooth

import re
from pynq.lib.rgbled  import RGBLED

RGBLEDS_XGPIO_OFFSET = 0
RGBLEDS_START_INDEX  = 4
RGB_CLEAR    = 0
RGB_BLUE     = 1
RGB_GREEN    = 2
RGB_CYAN     = 3
RGB_RED      = 4
RGB_MAGENTA  = 5
RGB_YELLOW   = 6
RGB_WHITE    = 7
MAX_LEDS_RGB = 2

rgbled_position = [4,5]
Delay0 = 1.0
Delay1 = 0.3
Delay2 = 0.1
color  = 0
    
 
def decode (word):
    sentence = ''
    if word is not None and word != '':
        for i in word:
            sentence = sentence + (chr(i)).strip()
        return  sentence.strip()
    else:
        return " "
    
def disconnect_uart ():
    logging.info ("def disconnect_uart ():")
    if 'uart' in globals():
        uart.close()
        
def process_cmd (cmd_uart):
    """
        Si nos envian el color azul:  conectamso Hivemqt y nos subscribimos topic para ver los eventos.
        Si nos envian el coor rojo: nps desconectamos Hivemqtt y nos desuscribimos topic
    """
    logging.info("def process_cmd (cmd_uart):")
    logging.info(cmd_uart)
    
    led_ix = int(cmd_uart[0])         #RGBLEDS_START_INDEX 
    color  = int(cmd_uart[1])        #Encendemos el testigo de app ok (verde led rgb)'
    app_r  = False
    app_connect = 0
    if (color == RGB_GREEN):
        # En este caso se escuchan los mensajes de la Cola HiveMQTT
        if  get_color_led_rgb (4) ==  2 :
            logging.info ("La App ya está arrancada!!!!.")
        else:
            #Encendemos el testigo de app ok (verde led rgb)
            logging.info('Encendemos el testigo de app ok (verde led rgb)')
            logging.info(led_ix)
            logging.info(color)
            led_rgb (led_ix,color)
    elif(color == RGB_RED):
        # Nos desconectamos de HiveMqtt y salimos de la app
        if  get_color_led_rgb (4) ==  2:
            #clean_led_stick ()
            #Encendemos el testigo de app nok(rojo led rgb)
            led_rgb (led_ix,color)
        else:
             logging.info ("La App no estaba en funcionamiento!!!!.")
    else:
        logging.info ("Comando Incorrecto!!!")
 
 
def init_uart (event_uart):
    
    patron = re.compile('\[(\d)*,(\d)*\]') #Patrón de entrada [4,5]
    #buttons  = [base.buttons[index] for index in range (4)]   

    for led in base.leds:
        led.on()
        
    for led in rgbled_position:
        led_rgb (led,color)
 
    while True:
        if event_uart.is_set():
            break
        logging.info("Leemos de la uart")
        word = recv_cmd (uart)
        sleep (Delay0)
        if word is not None and word != '':
            sentence = decode (word)

            logging.info(sentence)
            # Comprobamos si nos llega una petición a través de UART/bluetooth
            #---------------------------------------
            if patron.match(sentence) != None:
                logging.info("Comando Correcto")

                s =  [int(s) for s in re.findall(r'-?\d+\.?\d*', sentence)]

                logging.info (s)
                app_connect = process_cmd (s)

                #send_cmd (uart, 'App NOK')
            else:
                print ('Valor de entrada no es un comando valido: command =[',sentence,']')
        sleep(15)
        if  get_color_led_rgb (4) ==  4:
            break
    #--------------------------------- 

### 3.0 Test uart 

Nota: Debe tener un error el overlay base. Al tiempo de estar leyendo sobre el puerto, se produce una excepción que viene de las clases de MicroblazeLibrary. He perdido muchisimo tiempo con esto, y no puedo detectar porque pasa. Es en la lectura del puerto dispositivo_uart.read(respuesta, len_respuesta)


EXcepción:

File "/usr/lib/python3.10/threading.py", line 1009, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 946, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_1041/2832240255.py", line 88, in init_uart
  File "/tmp/ipykernel_1041/3418831147.py", line 5, in recv_cmd
  File "<string>", line 3, in wrapped
  File "/usr/local/share/pynq-venv/lib/python3.10/site-packages/pynq/lib/pynqmicroblaze/rpc.py", line 790, in _call_func
    return function(self._val, *args)
  File "<string>", line 3, in call
  File "/usr/local/share/pynq-venv/lib/python3.10/site-packages/pynq/lib/pynqmicroblaze/rpc.py", line 883, in call
    return_value, done = self._handle_stream(*args)
  File "/usr/local/share/pynq-venv/lib/python3.10/site-packages/pynq/lib/pynqmicroblaze/rpc.py", line 871, in _handle_stream
    return self.function.receive_response(self.stream, *args), True
  File "/usr/local/share/pynq-venv/lib/python3.10/site-packages/pynq/lib/pynqmicroblaze/rpc.py", line 551, in receive_response
    [
  File "/usr/local/share/pynq-venv/lib/python3.10/site-packages/pynq/lib/pynqmicroblaze/rpc.py", line 552, in <listcomp>
    f.param_decode(a, stream)
  File "/usr/local/share/pynq-venv/lib/python3.10/site-packages/pynq/lib/pynqmicroblaze/rpc.py", line 217, in param_decode
    assert length == len(old_val)
AssertionError

In [51]:
#event_uart  = multiprocessing.Event()
#led_rgb (4,2)
 
#r_uart = threading.Thread(target = init_uart,args=(event_uart,))
#r_uart.start ()

2023-01-05 09:45:13,873 root            INFO     Leemos de la uart
2023-01-05 09:45:14,992 root            INFO     ECH:OK    


Valor de entrada no es un comando valido: command =[ ECH:OK     ]


2023-01-05 09:45:30,012 root            INFO     Leemos de la uart
2023-01-05 09:45:31,121 root            INFO               


Valor de entrada no es un comando valido: command =[            ]


2023-01-05 09:45:46,142 root            INFO     Leemos de la uart
2023-01-05 09:45:47,250 root            INFO               


...

2023-01-05 09:55:10,661 root            INFO     Leemos de la uart
2023-01-05 09:55:11,769 root            INFO               


Valor de entrada no es un comando valido: command =[            ]


2023-01-05 09:55:26,790 root            INFO     Leemos de la uart
2023-01-05 09:55:27,899 root            INFO               


Valor de entrada no es un comando valido: command =[            ]


2023-01-05 09:55:42,913 root            INFO     Leemos de la uart
Exception in thread Thread-9 (init_uart):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1009, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 946, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_1041/2549632044.py", line 88, in init_uart
  File "/tmp/ipykernel_1041/1567291616.py", line 5, in recv_cmd
  File "<string>", line 3, in wrapped
  File "/usr/local/share/pynq-venv/lib/python3.10/site-packages/pynq/lib/pynqmicroblaze/rpc.py", line 790, in _call_func
    return function(self._val, *args)
  File "<string>", line 3, in call
  File "/usr/local/share/pynq-venv/lib/python3.10/site-packages/pynq/lib/pynqmicroblaze/rpc.py", line 883, in call
    return_value, done = self._handle_stream(*args)
  File "/usr/local/share/pynq-venv/lib/python3.10/site-packages/pynq/lib/pynqmicroblaze/rpc.py", line 871, in _handle_stream
    ret

In [ ]:
#if 'r_uart' in globals() or 'r_uart' in locals():
    #event_uart.set()  
    #r_uart.join ()

# 4. Hilo Simulador Eventos Distracccion

In [ ]:
def take_event (event):
    """
        EVENT_NFTR      = "NFTR" # Sin implementar
        EVENT_DANGPOS   = "DANGPOS" # Sin implementar
        #-- sensor Cam
        EVENT_DTBLINK   = "DTBLINK"
        EVENT_DTYAWN    = "DTYAWN"
        EVENT_DTSMOKING = "DTSMOKING"
        EVENT_DTDRINK   = "DTDRINK"
        EVENT_DTEATING  = "DTEATING"
        EVENT_DTMOBIL   = "DTMOBIL"
        #-- Sensor Temp
        EVENT_DTTEMP    = "DTTEMP"
        #-- Sensor Intensidad de Luz
        EVENT_DTLIGTH   = "DTLIGTH"

        #--- Actuadores

        EVENT_BUZZER    = "BUZZER"
        EVENT_LED_STICK = "LED_STICK"
        EVENT_SERVO     = "SERVO" 
        print(send_msg_json (EVENT_NFTR))
        print(send_msg_json (EVENT_TEMP,23))
        print(send_msg_json (EVENT_LIGHT_INT,23))
        
        
        logging.debug('Fin main script')
        logging.info('Fin main script')

    """
    msg = ""
    
    if event  == 'DTBLINK':
        logging.info ('DTBLINK')        
        msg = send_msg_json (event=EVENT_DTBLINK)
    elif event  == 'DTYAWN':
        logging.info ('DTYAWN')
        msg = send_msg_json (event=EVENT_DTYAWN)
    elif event  == 'DTSMOKING':
        logging.info ('DTSMOKING')
        msg = send_msg_json (event=EVENT_DTSMOKING)
    elif event  == 'DTDRINK':
        logging.info('DTDRINK')
        msg = send_msg_json (event=EVENT_DTDRINK)
    elif event  == 'DTEATING':
        logging.info('DTEATING')
        msg = send_msg_json (event=EVENT_DTEATING)
    elif event  == 'DTMOBIL':
        logging.info('DTMOBIL')
        msg = send_msg_json (event=EVENT_DTMOBIL)
    elif event  == 'DTLIGTH':
        logging.info('DTLIGTH')
        temp_scr.acquire()
        msg = send_msg_json (event=EVENT_DTLIGTH,vparam = temp_sensor ())
        temp_scr.release()
    elif event  == 'DTTEMP':
        logging.info('DTTEMP')
        intent_scr.acquire()
        msg = send_msg_json (event=EVENT_DTTEMP,vparam = ligth_intensity ())
        intent_scr.release()
    else:
        logging.info('error')
    
    logging.info(msg)
    
    return msg

list_events = ['DTBLINK','DTYAWN','DTSMOKING','DTDRINK','DTEATING','DTMOBIL','DTLIGTH','DTTEMP']


In [ ]:
def init_msg (event_msg):
    logging.info ("def init_msg ():")
 
    color       = 0
    app_connect = 0
    count        = 0
    t = 0
    #app_connect_hivemqtt_run ()
    sleep (2) 
    logging.info (list_events)
    
    mqtt_msg = False
    
    while get_color_led_rgb (4) !=  4:
        #print(' while get_color_led_rgb (4) ')
        if event_msg.is_set():
            break
        if  get_color_led_rgb (4) ==  2:
            if mqtt_msg == False:
                # Conecto al Cloud HiveMQTT y me suscribo
                print("me suscribo")
                sub_pynq = threading.Thread(target = subscribing_pynq,args=(client_hivemqtt,topic_sub_pynq))
                sub_pynq.start ()
                print("me suscribo 2")
                mqtt_msg = True
                
            #print(mqtt_msg)
            #print(client.connected_flag)
            if  mqtt_msg == True and client_hivemqtt.connected_flag == True:
                t=t+1
                ## TENEMOS UN EVENTO NUEVO QUE ENVIAMOS AL SERVIDOR
                #--------------------------------
                event = random.choice(list_events) 
                
                logging.info(event)
                msg = take_event (event)
                #logging.info ("msg")
                # Aquí realmente se pondria funcion proceado video detección objetos.
                # 
                #Creo un hilo para enviar el mensaje 
                pub_pynq = threading.Thread(target = main_pynq,args=(client_hivemqtt,topic_pub_pynq,msg))
                pub_pynq.start ()
            sleep (5)
            #-----------------------------------
        elif get_color_led_rgb (4) ==  4 and mqtt_msg == True:
            disconnect_hivemqtt()
            disconnect_client_cam(client_cam)
            logging.info ("Disconnected")
            break
        else:
            sleep (Delay0)
        if count == 100:
            count = 0
            clear_output(wait=True)
        else:
            count = count +1

### 4.0 Test Hilo eventos

In [30]:
#event_msg  = multiprocessing.Event()
#led_rgb (4,2)
#r_msg = threading.Thread(target = init_msg,args=(event_msg,))
#r_msg.start ()

In [ ]:
 
#if 'r_msg' in globals() or 'r_uart' in locals():
    #event_msg.set()  
    #r_msg.join ()

# 5. Hilo Procesado Video Streaming. Detección Somnolencia

In [ ]:
# Deteccion bostezos y somnolencia
 #--------------------------------------------------
    
def debug_ShowImg (img,hdmi_out):
    #cv2.imshow('debug_ShowImg : ',img)
    outframe = hdmi_out.newframe()
    outframe[0:480,0:640,:] = img [0:480,0:640,:]
    hdmi_out.writeframe(outframe)
    
    plt.imshow(img[:,:,[2,1,0]])
    plt.show()


    
def eye_aspect_ratio(eye):
    # Coordenadas verticales de la marca del ojo (X, Y)
    A = dist.euclidean(eye[1], eye[5])# Calcular la distancia euclidiana entre dos conjuntos
    B = dist.euclidean(eye[2], eye[4])
    # Calcular la distancia euclidiana entre niveles
    # Coordenadas horizontales de la marca del ojo (X, Y)
    C = dist.euclidean(eye[0], eye[3])
    # Cálculo de la relación de aspecto del ojo
    ear = (A + B) / (2.0 * C)
    # Devuelve la relación de aspecto de los ojos
    return ear
 
def mouth_aspect_ratio_(mouth):
    """
     Con esta formula cuando hablas lo detecta como bostezo. Mejor identificar más puntos de la boca
    """
    A = np.linalg.norm(mouth[2] - mouth[9])  # 51, 59
    B = np.linalg.norm(mouth[4] - mouth[7])  # 53, 57
    C = np.linalg.norm(mouth[0] - mouth[6])  # 49, 55
    mar = (A + B) / (2.0 * C)
    return mar

def mouth_aspect_ratio(mouth):
    # Compute the euclidean distances between the three sets
    # of vertical mouth landmarks (x, y)-coordinates
    A = np.linalg.norm(mouth[13] - mouth[19])
    B = np.linalg.norm(mouth[14] - mouth[18])
    C = np.linalg.norm(mouth[15] - mouth[17])

    # Compute the euclidean distance between the horizontal
    # mouth landmarks (x, y)-coordinates
    D = np.linalg.norm(mouth[12] - mouth[16])

    # Compute the mouth aspect ratio
    mar = (A + B + C) / (2 * D)

    # Return the mouth aspect ratio
    return mar


def run_drowsiness (event_cam):
    """
    EVENT_DANGPOS    = "DANGPOS" 
    EVENT_DROWSINESS = 'DROWSINESS'
    EVENT_DTBLINK    = "DTBLINK"
    EVENT_DTYAWN     = "DTYAWN"
    """
    global cap
    global hdmi_out
    # Definir dos constantes
    # Relación de aspecto del ojo y Umbral de parpadeo
    # Nota, el umbral de parpadeo se debe de configurar por persona. A la hora de registrarse debe medirse dicho umbral
    # cada persona tiene un umbral diferente. Además, la luz ambiente afecta a la medición, con lo que mejor todo en grises
    # Edtaria bien probar una mascara antes de realiar los calculos para evitar tener mediciones toxicas por ruido (FUTURO)
    EYE_AR_THRESH        = 0.3#0.2
    EYE_AR_CONSEC_FRAMES = 2 #3
    # Relación de aspecto de bostezo y Umbral de bostezo
    MAR_THRESH = 0.5 #0.5 
    MOUTH_AR_CONSEC_FRAMES = 2#3
    # Inicialice el contador de frames y el número total de parpadeos
    COUNTER = 0
    TOTAL = 0
    # Inicializar contador de frames y bostezo total
    mCOUNTER = 0
    mTOTAL = 0
    
    #Conador de conductor sin visión o nodetectado y umbral asociado. 
    countDrive = 0
    DRIVE_CONSEC_FRAMES = 40

    # monitor configuration: 640*480 @ 60Hz
    Mode = VideoMode(640,480,24)
    # monitor (output) frame buffer size
    hdmi_out = base.video.hdmi_out
    hdmi_out.configure(Mode,PIXEL_BGR)
    hdmi_out.start()
    # Nos creamos un publicador de eventos de la cam
    logging.info ("Voy a conectar el cliente publicador de eventos de la cam")
    #connect_client_cam ()
    
    # Inicializamos el detector de rostro (HOG) de DLIB, luego cree la predicción de hitos faciales. utilizaremos la libreria 
    logging.info ("[INFO] loading facial landmark predictor...")
    # Paso 1: Usaremos dlib.get_frontal_face_detector () para obtener el detector de posición de la cara
    detector = dlib.get_frontal_face_detector()
    # Paso 2: Usaremos dlib.shape_predictor para obtener el detector de posición de la función facial
    predictor = dlib.shape_predictor('./model/shape_predictor_68_face_landmarks.dat') #Datos red darcknet preentrenada

    # Paso 3: Obtenemos los valores estandares de índices de los signos faciales del ojo izquierdo y derecho por separado
    (lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
    (rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]
    (mStart, mEnd) = face_utils.FACIAL_LANDMARKS_IDXS["mouth"]

    # PAso 4: capturamos y obtenemos la cámara local cv2
    if  (('cap' in globals ()) and (cap is not None) and (cap.isOpened() == True)):
        logging.info ("--> Desactivo la captura anterior")
        cap.release()
    sistema = platform.system()
     
    os.environ["OPENCV_LOG_LEVEL"]="SILENT"   
    logging.info ("cv2.VideoCapture(0)")
    cap = cv2.VideoCapture(0)
    cap.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'))
    color = 0
    # Leemos frames a fremes el video en real time
    while color != 4:
        color = get_color_led_rgb (4)
        #logging.debug ("* color:",color)
        start_time = time.time() #--123---
        logging.debug ("* start_time:",start_time)
        if event_cam.is_set():
            break
        # Paso 5: bucle, leer la imagen, ampliar la dimensión de la imagen y escala de grises
        ret, frame = cap.read()
        if frame is None:
            continue
        frame = imutils.resize(frame, width=720)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        # Paso 6: Usamos  el detector (gris, 0) para la detección de la posición de la cara
        rects = detector(gray, 0)
        if len(rects) == 0:
            cv2.putText(frame, " Conductor NO DETECTADO!! RIESGO DE ACCIDENTE!!!", (20, 70),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (84, 255, 255), 2)
            #cv2.imshow("Frame", frame)
            debug_ShowImg (frame,hdmi_out)
            countDrive += 1
        else:
            countDrive = 0
         
        if countDrive > DRIVE_CONSEC_FRAMES: # DRIVE_CONSEC_FRAMES = 40 --- OJO!! hemos modificado los umbrales para desarrollo
            cv2.putText(frame, " RIESGO DE ACCIDENTE!!!", (20, 170),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (146, 43, 33), 2)
            #cv2.imshow("Frame", frame)
            debug_ShowImg (frame,hdmi_out)
            #Enviamos mensaje alerta conductor sin visión o no detectado
            #print("Voy a enviar mensaje")
            #send_cam_scr.acquire()
            m = [send_msg_json (event=EVENT_DANGPOS,clientid=client_cam_id)]
            send_msg = threading.Thread(target = send_cloud_cam_mseg,args=(m,))
            send_msg.start ()# Enviamos alerta de somnolencia  
            #send_cam_scr.release() 
            #send_cloud_cam_mseg (m)
            #print("Voy a enviar mensaje")
            mmsg = f'event:SLEEP, total:{TOTAL},mtotal:{mTOTAL}'
            logging.critical (mmsg)
            
        # Realmente el Rects tenemos más de una cara. En paso futuro hay que extraer aquella que corresponde al conductor (Si la hubiese) y compararla
        # con la almacenada en el sistema (Cuando se realizo el resgitro de rasgos faciales).
        # Paso 7: repita la información de la posición facial y use el predictor (gris, rect) para obtener la información de posición de la función facial
        for rect in rects:
            shape = predictor(gray, rect)

            shape = face_utils.shape_to_np(shape)

            # Paso 9: Extraemos las coordenadas de los ojos izquierdo y derecho
            leftEye = shape[lStart:lEnd]
            rightEye = shape[rStart:rEnd]
            
            # Paso 10: Extraemos las coordenadas de boca
            mouth = shape[mStart:mEnd]


            # Paso 11: el constructor calcula el valor EAR de los ojos izquierdo y derecho. Utiliamos  el valor promedio como EAR final
            leftEAR = eye_aspect_ratio(leftEye)
            rightEAR = eye_aspect_ratio(rightEye)
            ear = (leftEAR + rightEAR) / 2.0
            # Paso 12:REalizamos el mismo paso para el caso de la boca
            mar = mouth_aspect_ratio(mouth)

            # Paso 13: Usamos cv2.convexHull para obtener la posición del casco convexo, y con drawContours dibujamos la posición del contorno 
            leftEyeHull  = cv2.convexHull(leftEye)
            rightEyeHull = cv2.convexHull(rightEye)
            
            cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
            cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)
            
            # Paso 14: Idem con la boca
            mouthHull = cv2.convexHull(mouth)
            cv2.drawContours(frame, [mouthHull], -1, (0, 255, 0), 1)

            # Paso 15:  Recuadramos la cara que estamos evaluando (EN futuro, debemos de compararla con al del conductor registrado)
            left = rect.left()
            top = rect.top()
            right = rect.right()
            bottom = rect.bottom()
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 3)    

            '''
                A partir de aqui realizamos los cálculos de fatiga.
            '''
            # PAso 16: Si el valor de ear calculado es menor al umbral establecido para dicho conductor entonces incrementamos nñumero de parpadeos
            if ear < EYE_AR_THRESH:# Relación de aspecto del ojo: el normal 0.2, para ojos pequeños 0.1
                COUNTER += 1
            else:
                # Si es inferior al umbral por EYE_AR_CONSEC_FRAMES veces consecutivas, significa que se ha realizado una actividad de parpadeo
                if COUNTER >= EYE_AR_CONSEC_FRAMES:# Umbral: 3, OJO!! hemos modificado los umbrales para desarrollo
                    TOTAL += 1 # Nos indica las veces que ha parpadeado cada EYE_AR_CONSEC_FRAMES consecutivos
                    #print("--------------------Voy a enviar mensaje")
                    #send_cam_scr.acquire()
                    m = [send_msg_json (event=EVENT_DTBLINK,clientid=client_cam_id)] 
                    send_msg =  threading.Thread(target = send_cloud_cam_mseg,args=(m,))
                    send_msg.start ()# Enviamos alerta de parpadeo  
                    #send_cam_scr.release() 
                    #send_cloud_cam_mseg (m)
                    #print("--------------------Voy a enviar mensaje")
                    mmsg = f'event:BLINK,total:{TOTAL},mtotal:{mTOTAL}'
                    logging.critical (mmsg)
                # Restablecer contador de marco de ojo
                COUNTER = 0

            # Paso 14: Sacamos por pantalla utilizando cv2.putText el número de parpadeos y el valor del ear obtenido en el frame
            cv2.putText(frame, "Faces: {}".format(len(rects)), (10, 30),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            cv2.putText(frame, "Blinks: {}".format(TOTAL), (150, 30),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            cv2.putText(frame, "COUNTER: {}".format(COUNTER), (300, 30),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2) 
            cv2.putText(frame, "EAR: {:.2f}".format(ear), (455, 30),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            mmsg = f'event:INFO,Blinks:{TOTAL},COUNTER:{COUNTER},EAR:{ear}'
            logging.critical (mmsg)
            
            '''
Calculamos la puntuación de boca abierta, si es menor que el umbral, se incrementa en 1; si es menor que el umbral por MOUTH_AR_CONSEC_FRAMES veces consecutivas, significa que se produce un bostezo, y el mismo bostezo es de aproximadamente MOUTH_AR_CONSEC_FRAMES frames.
            '''
            # Paso 15: Del mismo modo comprobamos si se ha producio un bostezo    
            if mar > MAR_THRESH:# Umbral de boca abierta 0.5
                mCOUNTER += 1
                cv2.putText(frame, "Yawning!", (10, 60),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                mmsg = f'event:Yawning,total:{TOTAL},mtotal:{mTOTAL}'
                logging.critical (mmsg)
            else:
                # Si es inferior al umbral por 3 veceprint("--------------------Voy a enviar mensaje")s consecutivas, significa un bostezo
                if mCOUNTER >= MOUTH_AR_CONSEC_FRAMES:# Umbral: 3 --- OJO!! hemos modificado los umbrales para desarrollo
                    mTOTAL += 1
                    mmsg = f'event:YAWN,total:{TOTAL},mtotal:{mTOTAL}'
                    logging.critical (mmsg)
                    #print("--------------------Voy a enviar mensaje")
                    #send_cam_scr.acquire()
                    m = [send_msg_json (event=EVENT_DTYAWN,clientid=client_cam_id)] 
                    send_msg =  threading.Thread(target = send_cloud_cam_mseg,args=(m,))
                    send_msg.start ()# Enviamos alerta de bostezo 
                    #send_cam_scr.release() 
                    #send_cloud_cam_mseg (m)
                    #print("--------------------Voy a enviar mensaje")
                # Restablecer contador de marco de boca
                mCOUNTER = 0
            cv2.putText(frame, "Yawning: {}".format(mTOTAL), (150, 60),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            cv2.putText(frame, "mCOUNTER: {}".format(mCOUNTER), (300, 60),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2) 
            cv2.putText(frame, "MAR: {:.2f}".format(mar), (480, 60),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

            #  Paso 16: operación de dibujo, 68 identificación de puntos de características de dlib (Futuro poner mediapipe en la pynq)
            for (x, y) in shape:
                cv2.circle(frame, (x, y), 1, (0, 0, 255), -1)

           #print('Relación de aspecto en tiempo real de la boca: {: .2f}'.format(mar)+"\ topen tu boca:"+str([False,True][mar > MAR_THRESH]))
           # print('Relación de aspecto en tiempo real de los ojos: {: .2f}'.format(ear)+"\ tBlink:"+str([False,True][COUNTER>=1]))

        # Determinar consejos de fatiga
        # TOTAL >= 20 or mTOTAL>=10:
        if TOTAL >= 50 or mTOTAL>=20:
            cv2.putText(frame, "SLEEP!!!. Riesgo Acidente!!!", (100, 200),cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)
            #Enviamos mensaje alerta microsueño y lo guardamos en el log
            #print("Voy a enviar mensaje")
            #send_cam_scr.acquire()
            m = [send_msg_json (event=EVENT_DROWSINESS,clientid=client_cam_id)]
            send_msg =  threading.Thread(target = send_cloud_cam_mseg,args=(m,))
            send_msg.start ()# Enviamos alerta de somnolencia  
            #send_cam_scr.release() 
            #send_cloud_cam_mseg (client_cam,m)
            #print("Voy a enviar mensaje")
            mmsg = f'event:SLEEP, total:{TOTAL},mtotal:{mTOTAL}'
            logging.critical (mmsg)
           
        # Presione q para salir
        #cv2.putText(frame, "Press 'q': Quit", (20, 500),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (84, 255, 159), 2)
        # Muestra de ventana con opencv
        #cv2.imshow("Frame", frame)
        debug_ShowImg (frame,hdmi_out)
        elapsed_time = time.time() - start_time ### ---123--
        mm = f"Elapsed time: {elapsed_time} seconds."
        logging.debug(mm)
        # if the `q` key was pressed, break from the loop
        #if cv2.waitKey(1) & 0xFF == ord('q'):
            #break

    # Release camera release camera
    cap.release()
    # do a bit of cleanup
    cv2.destroyAllWindows()
    hdmi_out.stop()
    del hdmi_out


### 5.0 Test Proceso Somnolencia

In [ ]:
m = [send_msg_json (event=EVENT_DROWSINESS,clientid=client_cam_id)]
print(m)
send_msg =  threading.Thread(target = send_cloud_cam_mseg,args=(client_cam,m))
send_msg.start ()# Enviamos alerta de somnolencia  
 

In [ ]:
m = [send_msg_json (event=EVENT_DTYAWN,clientid=client_cam_id)] 
send_msg =  threading.Thread(target = send_cloud_cam_mseg,args=(client_cam,m))
send_msg.start ()# Enviamos alerta de bostezo 

In [ ]:
m = [send_msg_json (event=EVENT_DTBLINK,clientid=client_cam_id)] 
send_msg =  threading.Thread(target = send_cloud_cam_mseg,args=(client_cam,m))
send_msg.start ()# Enviamos alerta de parpadeo  

In [ ]:
#m = [send_msg_json (event=EVENT_DANGPOS,clientid=client_cam_id)]
#send_msg = threading.Thread(target = send_cloud_cam_mseg,args=(client_cam,m))
#send_msg.start ()# Enviamos alerta de somnolencia  

In [ ]:
#event_cam  = multiprocessing.Event()
#led_rgb (4,2)
#r_cam = threading.Thread(target = run_drowsiness,args=(event_cam,))
#r_cam.start ()

In [ ]:
#event_cam  = multiprocessing.Event() 
#disconnect_client_cam(client_cam)
 
#event_cam.set()  
#r_cam.join ()

In [ ]:
#cap.release()
# do a bit of cleanup
#cv2.destroyAllWindows()
#hdmi_out.stop()
#del hdmi_out

# 6.  Hilo de arranque de la APP en la PYNQ

In [ ]:
def init_app_con_somnolencia ():
    led_rgb(4,0)
    event_uart = multiprocessing.Event() 
    event_msg  = multiprocessing.Event()
    event_cam  = multiprocessing.Event() 
    #Hilo dedicado eventos uart/bluetooth (conexión/desconexión)
    r_uart = threading.Thread(target = init_uart,args=(event_uart,))
    r_uart.start ()
    
    #Hilo dedicado a mensajes de eventos 
    r_msg = threading.Thread(target = init_msg,args=(event_msg,))
    r_msg.start ()  
    color = 0
    cam_ok = False
    while get_buttons()==0:
        color = get_color_led_rgb (4)
        logging.info(color)
        if color == 2 and cam_ok == False:
            logging.info("Arrancamos la cam de la pynq")
            cam_ok = True
            r_cam = threading.Thread(target = run_drowsiness,args=(event_cam,))
            r_cam.start ()
        if color == 4:
            cam_ok = False
            break
        sleep (1)  
        
    
    print('press buttons!!!!')
    if 'r_cam' in globals() or 'r_cam' in locals():
        event_cam.set()  
        r_cam.join ()
    if 'r_msg' in globals() or 'r_msg' in locals():
        event_msg.set()  
        r_msg.join ()
    if 'r_uart' in globals() or 'r_uart' in locals():
        event_uart.set() 
        r_uart.join ()
    disconnect_hivemqtt()
    disconnect_client_cam(client_cam)
    cap.release()
    # do a bit of cleanup
    cv2.destroyAllWindows()
    hdmi_out.stop()
    del hdmi_out

def close_app ():
    logging.debug("def close_app ():")
    disconnect_hivemqtt()
    disconnect_client_cam()
    if get_color_led_rgb (4) == 2:
        led_rgb(4,4)
    
def init_app ():
    
    event_uart = multiprocessing.Event() 
    event_msg  = multiprocessing.Event()
    led_rgb(4,0)
    #Hilo dedicado eventos uart/bluetooth (conexión/desconexión)
    r_uart = threading.Thread(target = init_uart,args=(event_uart,))
    r_uart.start ()
    
    #Hilo dedicado a mensajes de eventos 
    r_msg = threading.Thread(target = init_msg,args=(event_msg,))
    r_msg.start ()  
    color = 0

    while get_buttons()==0:
        color = get_color_led_rgb (4)
        logging.info(color)

        if color == 4:
            break
        sleep (4)  
        
    
    print('press buttons!!!!')
    if 'r_msg' in globals() or 'r_msg' in locals():
        event_msg.set()  
        r_msg.join ()
    if 'r_uart' in globals() or 'r_uart' in locals():
        event_uart.set() 
        r_uart.join ()
    disconnect_hivemqtt()
    disconnect_client_cam(client_cam)
    cap.release()
    # do a bit of cleanup
    cv2.destroyAllWindows()
    hdmi_out.stop()
    del hdmi_out

# 7. Arranque de la APP

## 7.1 Arranque sin control somolencia en pynq

In [ ]:
init_app ()  # Con detección somnolencia en PC

## 7.2 Arranque con control de somnolencia en pynq

In [ ]:
init_app_con_somnolencia ()  # Con detección somnolencia con la pynq